Import Libraries & Load the Validated Datasets

In [1]:
import pandas as pd

# Load validated datasets
full_data = pd.read_csv('data/combined_validated.csv', encoding='ISO-8859-1')
incremental_data = pd.read_csv('data/incremental_data.csv', encoding='ISO-8859-1')

- We load the validated full and incremental datasets to apply transformations.


Check Initial Overview

In [2]:
print("Full Data Shape:", full_data.shape)
print("Incremental Data Shape:", incremental_data.shape)
print(full_data.head())

Full Data Shape: (536641, 8)
Incremental Data Shape: (25525, 8)
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2  2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


- This confirms both datasets loaded correctly.

# APPLY TRANSFORMATIONS

Transformation 1: Handle Missing Values (Cleaning)

In [3]:
# Before
print("Missing values before cleaning:\n", full_data.isnull().sum())

# Drop rows with missing CustomerID
full_data = full_data.dropna(subset=['CustomerID'])
incremental_data = incremental_data.dropna(subset=['CustomerID'])

# After
print("\nMissing values after cleaning:\n", full_data.isnull().sum())

Missing values before cleaning:
 InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135037
Country             0
dtype: int64

Missing values after cleaning:
 InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


- Removes incomplete rows to ensure valid customer transactions.

Transformation 2: Remove Duplicates (Cleaning)

In [4]:
# Before
print("Duplicates before:", full_data.duplicated().sum())

# Remove duplicates
full_data.drop_duplicates(inplace=True)
incremental_data.drop_duplicates(inplace=True)

# After
print("Duplicates after:", full_data.duplicated().sum())

Duplicates before: 0
Duplicates after: 0


- Ensures unique transaction records

Transformation 3: Convert and Format Dates (Standardization)

In [5]:
# Before
print("Date type before:", full_data['InvoiceDate'].dtype)

# Convert InvoiceDate to datetime
full_data['InvoiceDate'] = pd.to_datetime(full_data['InvoiceDate'], errors='coerce')
incremental_data['InvoiceDate'] = pd.to_datetime(incremental_data['InvoiceDate'], errors='coerce')

# After
print("Date type after:", full_data['InvoiceDate'].dtype)


Date type before: object
Date type after: datetime64[ns]


- Converts InvoiceDate from string/object to proper datetime type.

Transformation 4: Add Derived Column (Enrichment)

In [6]:
# Before
print(full_data.head(2))

# Create a new column: total_cost = Quantity * UnitPrice
full_data['TotalCost'] = full_data['Quantity'] * full_data['UnitPrice']
incremental_data['TotalCost'] = incremental_data['Quantity'] * incremental_data['UnitPrice']

# After
print(full_data[['Quantity', 'UnitPrice', 'TotalCost']].head(2))

  InvoiceNo StockCode                         Description  Quantity  \
0    536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                 WHITE METAL LANTERN         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
   Quantity  UnitPrice  TotalCost
0         6       2.55      15.30
1         6       3.39      20.34


- Adds a useful metric for business analysis (sales amount per transaction).

Transformation 5: Filter Out Irrelevant or Invalid Records (Structural)

In [7]:
# Before
print("Before filtering:", full_data.shape)

# Keep only positive Quantity and UnitPrice
full_data = full_data[(full_data['Quantity'] > 0) & (full_data['UnitPrice'] > 0)]
incremental_data = incremental_data[(incremental_data['Quantity'] > 0) & (incremental_data['UnitPrice'] > 0)]

# After
print("After filtering:", full_data.shape)


Before filtering: (401604, 9)
After filtering: (392692, 9)


- Removes invalid transactions (negative or zero values).

Transformation 6 : Categorize Sales into Brackets

In [8]:
# Create sales categories
bins = [0, 50, 200, 500, 1000, full_data['TotalCost'].max()]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
full_data['SalesCategory'] = pd.cut(full_data['TotalCost'], bins=bins, labels=labels, include_lowest=True)
incremental_data['SalesCategory'] = pd.cut(incremental_data['TotalCost'], bins=bins, labels=labels, include_lowest=True)

print(full_data[['TotalCost', 'SalesCategory']].head(5))

   TotalCost SalesCategory
0      15.30      Very Low
1      20.34      Very Low
2      22.00      Very Low
3      20.34      Very Low
4      20.34      Very Low


- Adds meaningful business segmentation — useful for reports and dashboards.

Save Transformed Data

In [9]:
# Save transformed datasets
full_data.to_csv('transformed/transformed_full.csv', index=False)
incremental_data.to_csv('transformed/transformed_incremental.csv', index=False)

print("✅ Transformed datasets saved successfully.")


✅ Transformed datasets saved successfully.
